In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

In [ ]:
# Jenny's codes start here
vaccine_county = pd.read_csv("Clean_data/vaccine_us_county_clean.csv",low_memory=False)

In [ ]:
vaccine_county_rename = vaccine_county.rename(columns={"Recip_County": "County", "Series_Complete_Pop_Pct": "Vaccination_Complete_Pct","Recip_State":"State"})
vaccine_county_rename.head()

In [ ]:
svi_df = pd.read_csv("Resource/04_siv_county.csv",low_memory=False)
svi_df = svi_df.dropna()
svi_df_rename = svi_df.rename(columns={"FIPS Code": "FIPS", "County Name": "County"})

In [ ]:
svi_df_rename['County'] = svi_df_rename['County'].str.split(',', expand=True)[0]

In [ ]:
vaccine_svi_df = pd.merge(vaccine_county_rename, svi_df_rename, on='FIPS')
vaccine_svi_df = vaccine_svi_df[['Date','FIPS','County_x','State_x','Vaccination_Complete_Pct','Metro_status','Social Vulnerability Index (SVI)','SVI Category']] 
vaccine_svi_df = vaccine_svi_df.rename(columns={"County_x": "County","State_x": "State"})

In [ ]:
# Jenny's codes end here
vaccine_svi_df.to_csv("Clean_data/vaccine_svi_df.csv", encoding="utf-8", index=False)